## Create Azure Machine Learning datasets for Predictive Maintenance 

Azure Machine Learning datasets can be extremely useful for your local or remote experiments. In this notebook, we will do the following things.

1. Configure workspace using credentials for Azure subscription
2. Download the dataset from ADLS Gen2
3. Upload the featured dataset into the default datastore in Azure
4. Register the featured dataset into Azure

## Configure workspace using credentials for Azure subscription

As part of the setup you have already created a Workspace. To run AutoML, you also need to create an Experiment. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [7]:
# Install the required package
!pip install azure-storage-blob==2.1.0

from azureml.core import Workspace

# Importing user defined config
import config

# Import the subscription details as below to access the resources
subscription_id=config.subscription_id
resource_group=config.resource_group
workspace_name=config.workspace_name

try:
    workspace = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    workspace.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Workspace configuration succeeded. Skip the workspace creation steps below


## Download the  dataset from ADLS Gen2

In [11]:
## setting up the credentials for ADLS Gen2
import os
from azure.storage.blob import BlockBlobService

# setting up blob storage configs
STORAGE_ACCOUNT_NAME = config.STORAGE_ACCOUNT_NAME
STORAGE_ACCOUNT_ACCESS_KEY = config.STORAGE_ACCOUNT_ACCESS_KEY
STORAGE_CONTAINER_NAME = "azureml-mfg"

blob_service = BlockBlobService(STORAGE_ACCOUNT_NAME, STORAGE_ACCOUNT_ACCESS_KEY) 

output_file_path=os.path.join(os.getcwd(),"data", "mfg_pdm.csv")
output_blob_file= "mfg_pdm.csv"

# Create a project_folder if it doesn't exist
if not os.path.isdir('data'):
 os.mkdir('data')

# uploading the csv to  the ADLSGen2 storage container
blob_service.get_blob_to_path(STORAGE_CONTAINER_NAME, output_blob_file,output_file_path)



## Upload the featured dataset into the default datastore in Azure

In [15]:
from sklearn import datasets
from azureml.core.dataset import Dataset
from scipy import sparse
import os 
 
# Create a project_folder if it doesn't exist
if not os.path.isdir('data'):
 os.mkdir('data')
 
 
ds = workspace.get_default_datastore()
ds.upload(src_dir='./data', target_path='mfgdata', overwrite=True, show_progress=True)
 
final_df = Dataset.Tabular.from_delimited_files(path=ds.path('mfgdata/mfg_pdm.csv'))

Uploading an estimated of 1 files
Uploading ./data/mfg_pdm.csv
Uploaded ./data/mfg_pdm.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## Register the featured dataset into Azure

In [17]:
# train_data_registered = Dataset.get_by_name(amlworkspace,"train_data",version='latest')
#train_data_registered.unregister_all_versions()

train_data_registered = final_df.register(workspace=workspace,
                                 name='pdmmfg',
                                 description='Synapse Mfg data',
                                 tags= {'type': 'Mfg', 'date':'2020'},
                                 create_new_version=False)